In [18]:
from AnthropicWrapper import ClaudeChat
from HumeWrapper import HumeSentimentAnalyzer
#from ConversationVerifier import ConversationVerifier
from dotenv import load_dotenv
from joblib import load
import os

In [11]:
# -- Global Variables --
chatlog = load('conversation.joblib') # Loading in the list of dicts

In [4]:
dotenv_path = os.path.join(os.path.dirname(os.getcwd()), ".env")
load_dotenv(dotenv_path)

True

In [ ]:
# Initialising the Sentiment Analyser
sentiment_analyser = HumeSentimentAnalyzer(os.getenv("HUME_API_KEY"))

In [ ]:
# Initialising Sentiment Summariser
chat_model_name = "claude-3-5-sonnet-20240620"

system_prompt = "Claude, write me a sonnet about the beauty of the universe."

sentiment_analyser = ClaudeChat(chat_model_name, system_prompt)

In [ ]:
# Initialising verifier
# TBD

In [ ]:
result = sentiment_analyser.analyze_audio(os.path.join(script_directory, "interview_audio.wav"))
